In [25]:
import datetime
import pandas as pd
import tushare as ts
import sys
import requests

In [3]:
# tushare_token = sys.argv[1]
tushare_token = 'c0641675f20fa1b0c0787235e132a60a1242a89bdf953952773d71e5'
ts.set_token(tushare_token)

pro = ts.pro_api(tushare_token)

In [99]:
ts_token2 = 'd4815851268468e0b1ec29de65b0d31d8e0d3e0c04a1d89fe04a0120'

ts.set_token(ts_token2)

pro = ts.pro_api(ts_token2)

## 定义股票价格四线预警类

In [128]:
class stock_ma(object):
    def __init__(self,stock_code):
        self.stock_code = stock_code
        self.today = datetime.datetime.today().date()
    
    # 获取实时数据
    def get_real_info(self):
        df_real = ts.get_realtime_quotes(self.stock_code.split('.')[0])
        df_real['price'] = df_real['price'].astype(float)
        df_real['pre_close'] = df_real['pre_close'].astype(float)
        real_info = df_real.iloc[0].to_dict()
        real_info['return'] = (real_info['price'] - real_info['pre_close'])/real_info['pre_close']
        df_real['date'] = pd.to_datetime(df_real['date'])
        df_real['date_week'] = df_real['date'].apply(lambda x:x + datetime.timedelta(4 - x.weekday()))
        data_real = df_real[['date','date_week','price']]
        data_real.columns = ['date','date_week','close']
        return real_info,data_real
    
    # 获取历史数据
    def get_history_info(self):
        real_info,data_real = self.get_real_info()
        
        start_date = (self.today+datetime.timedelta(-500)).strftime('%Y%m%d')
        end_date = ''.join(real_info['date'].split('-'))
        df = ts.pro_bar(ts_code=self.stock_code, adj='qfq', start_date=start_date, end_date=end_date,ma=[5, 20, 50])
        df['date'] = df['trade_date'].apply(lambda x:datetime.datetime.strptime(x,'%Y%m%d'))
        # 添加当周周五，计算周均线
        df['date_week'] = df['date'].apply(lambda x:x + datetime.timedelta(4 - x.weekday()))
        data_history = df.sort_values(by=['date'],ascending=False).loc[1:][['date','date_week','close']]
        return data_history
    
    # 合并实时数据与历史数据
    def merge_data(self):
        real_info,data_real = self.get_real_info()
        data_history = self.get_history_info()
        data = pd.concat([data_real,data_history])
        return data
    
    # 计算5日、21日、60日、5周线、21周线、60周线
    def calc_ma(self):
        data = self.merge_data()
        data['ma5'] = data['close'][::-1].rolling(5).mean()[::-1]
        data['ma21'] = data['close'][::-1].rolling(21).mean()[::-1]
        data['ma60'] = data['close'][::-1].rolling(60).mean()[::-1]
        ma21_week = data.groupby('date_week')['close'].first().rolling(21).mean().to_frame(name = 'ma21_week').reset_index()
        ma60_week = data.groupby('date_week')['close'].first().rolling(60).mean().to_frame(name = 'ma60_week').reset_index()
        data_21week = pd.merge(data,ma21_week,on='date_week')
        data_60week = pd.merge(data_21week,ma60_week)
        return data_60week
    
    # 输出最新价及4线
    def print_info(self):
        data_60week = self.calc_ma()
        real_info,data_real = self.get_real_info()
        ma_info = data_60week.iloc[0].to_dict()
        ma_ls = [ma_info['ma21'],ma_info['ma60'],ma_info['ma21_week'],ma_info['ma60_week']]
        flag_ls = [float(real_info['price']) >= i for i in ma_ls]
        price_dis = sum([(real_info['price']-i)/i for i in ma_ls])/4
        ma_content = '|'.join([f'`{x:.2f}`' if y else f'{x:.2f}' for (x,y) in list(zip(ma_ls,flag_ls))])
#         mark_ls = ['最新价在此上方' if flag  else '最新价在此下方' for flag in flag_ls]
#         print_info = f'''{real_info["time"]}|{real_info["name"]}({real_info["code"]})|{real_info["price"]}|处于{sum(flag_ls)}线上方|<font color={mark_ls[0]}>{ma_info["ma21"]:.2f}</font>|<font color={mark_ls[1]}>{ma_info["ma60"]:.2f}</font>|<font color={mark_ls[2]}>{ma_info["ma21_week"]:.2f}</font>|<font color={mark_ls[3]}>{ma_info["ma60_week"]:.2f}</font>'''
        print_info = f'{real_info["time"]}|{real_info["name"]}|{real_info["code"]}|`{real_info["price"]}`|{real_info["return"]:.2%}|处于`{sum(flag_ls)}`线上方|{price_dis:.2%}|{ma_content}'
#         print_info = f'{real_info["time"]}, {real_info["name"]}({real_info["code"]}), 最新价{real_info["price"]}, 处于{sum(flag_ls)}线上方\n\n21日线|60日线|21周线|60周线\n---|---|---|---\n{ma_info["ma21"]:.2f}|{ma_info["ma60"]:.2f}|{ma_info["ma21_week"]:.2f}|{ma_info["ma60_week"]:.2f}'
        ma_nums = sum(flag_ls)
        return ma_nums,print_info
        

In [100]:
df = pro.query('daily', start_date='20180701', end_date='20180718')

### 获取当前股票列表

In [48]:
data = pro.stock_basic(exchange='', list_status='L', fields='ts_code,symbol,name,area,industry,list_date')

data['ipo_date'] = data['list_date'].apply(lambda x:datetime.datetime.strptime(x,'%Y%m%d'))

# 过滤满足时间的股票

min_date = datetime.datetime.today().date() + datetime.timedelta(-500)

filter_data = data[data['ipo_date'].apply(lambda x:x <= min_date)] 

In [53]:
filter_data.shape

(3546, 7)

In [67]:
# 中小板股票
stocks_sz = filter_data[filter_data['ts_code'].apply(lambda x:x.startswith('0'))]

In [115]:
# 创业板股票
stocks_szc = filter_data[filter_data['ts_code'].apply(lambda x:x.startswith('3'))].reset_index()

In [129]:
stock = stocks_szc['ts_code'][0]

In [130]:
stock_ma(stock).print_info()

(2,
 '15:00:03|特锐德|300001|`19.87`|-0.40%|处于`2`线上方|-2.12%|`19.50`|21.48|21.14|`19.27`')

### 获取所有创业板股票一段时间内的数据

In [131]:
# 测试10只创业板股票

stocks_szc_ma = [stock_ma(stock).print_info() for stock in stocks_szc['ts_code'][:10]]

In [132]:
pick_stocks = [i for i in stocks_szc_ma if i[0]>=3]

In [133]:
len(pick_stocks)

5

In [134]:
msg_ls = [i[1] for i in pick_stocks]

In [135]:
header = '时间|名称|代码|最新价|涨跌幅|4线位置|距离|21日线|60日线|21周线|60周线\n---|---|---|---|---|---|---|---|---'

table = header + '\n' + '\n'.join(msg_ls)

In [136]:
print(table)

时间|名称|代码|最新价|涨跌幅|4线位置|距离|21日线|60日线|21周线|60周线
---|---|---|---|---|---|---|---|---
15:00:03|神州泰岳|300002|`5.9`|6.88%|处于`4`线上方|30.42%|`5.66`|`4.50`|`4.22`|`4.03`
15:00:03|乐普医疗|300003|`38.76`|-2.12%|处于`3`线上方|9.44%|39.14|`37.59`|`36.41`|`30.00`
15:00:03|天海防务|300008|`2.89`|1.05%|处于`3`线上方|-0.01%|`2.86`|`2.73`|`2.81`|3.21
15:00:03|安科生物|300009|`17.45`|2.11%|处于`4`线上方|13.36%|`15.29`|`15.21`|`15.46`|`15.62`
15:00:03|立思辰|300010|`18.11`|2.90%|处于`4`线上方|20.26%|`16.46`|`17.27`|`16.02`|`11.83`
